In [ ]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import lyricsgenius

In [ ]:
def create_dataset(genres, count=5):
    '''
    Create dataset and save it to CSV file.
    Parameters:
        genres (list): List of genre names.
        count (int): Desired number of observations from each genre.
    '''
    k = int(count / 50)
    l = count % 50
    for i in range(k + 1):
        lyrics = []
        genre_names = []
        if i == k:
            count = l
            if count == 0:
                break
        else:
            count = 50
        for genre in genres:
            try:
                df = pd.DataFrame(sp.search(q=f'genre:{genre}', limit=count, type='track', market='US', offset=(i * 50))['tracks']['items'])[['artists', 'name']]
                df.artists = df.artists.apply(lambda x: x[0]['name'])
                for index, song in df.iterrows():
                    song_lyrics = genius.search_song(song['name'], song['artists'])
                    if song_lyrics is None:
                        continue
                    song_lyrics = song_lyrics.to_text()
                    lyrics.append(song_lyrics[song_lyrics.find('Lyrics') + 6:])
                    genre_names.append(genre)
            except:
                continue
        df = pd.DataFrame({
            'lyrics': lyrics,
            'genre': genre_names
        })
        df = df.sample(frac=1).reset_index(drop=True)
        df.to_csv('data/raw/dataset.csv', index=False, mode='a', header=False)

In [ ]:
# Needed credentials

cid_spotify = ''
secret_spotify = ''
cat_genius = ''

In [ ]:
client_credentials_manager = SpotifyClientCredentials(client_id=cid_spotify, client_secret=secret_spotify)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager, requests_timeout=10, retries=10)

genius = lyricsgenius.Genius(cat_genius, timeout=10, retries=10)

In [ ]:
# Possible genres to choose from

print(list(sp.recommendation_genre_seeds()['genres']))

In [ ]:
# Creation of a dataset with 5 genres 1000 observations each

create_dataset(['country', 'hip-hop', 'metal', 'pop', 'rock'], 1000)